# Make a function to build an inverted index and a tfidf vectorizer given tokenized text.

In [47]:
from collections import defaultdict

In [48]:
# TODOS
# Figure out how to use inverted index with tfidf vectorizer (may have to make manual tfidf vectorizer)
# Try using n-grams parameter and see how it affects results

In [65]:
def get_inverted_index(list_of_jokes, include_title, include_post):
    data = [(dt['title'] + ' ' + dt['post'] if include_post else dt['title']) if include_title else 
            (dt['post'] if include_post else '') for dt in list_of_jokes]
    tok_data = [dt.split(' ') for dt in data]
    return build_inverted_index(tok_data)

def build_inverted_index(list_of_toks_lists):
    """ Builds an inverted index from the messages.
    
    Arguments
    =========
     
    msgs: list of dicts.
        Each message in this list already has a 'toks'
        field that contains the tokenized message.
    
    Returns
    =======
    
    index: dict
        For each term, the index contains a list of
        tuples (doc_id, count_of_term_in_doc):
        index[term] = [(d1, tf1), (d2, tf2), ...]
        
    Example
    =======
    
    >> test_idx = build_inverted_index([
    ...    {'toks': ['to', 'be', 'or', 'not', 'to', 'be']},
    ...    {'toks': ['do', 'be', 'do', 'be', 'do']}])
    
    >> test_idx['be']
    [(0, 2), (1, 2)]
    
    >> test_idx['not']
    [(0, 1)]
    
    """
    # term --> tuple
    index = defaultdict(list)
    #for m in msgs:
    for doc_id in range(0, len(list_of_toks_lists)):
        term_to_count = defaultdict(int)
        for tok in list_of_toks_lists[doc_id]:
            term_to_count[tok] += 1
        for t, cnt in term_to_count.iteritems():
            index[t].append((doc_id, cnt))
    return index    

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

# input: list of dictionaries, where each dict is a joke.  The dict must have 'text' as a key. 
# include_title and include_text are boolean flags -- important because need to determine whether we want to include 
# just title for the text of the joke, or just the actual post, or both. (obviously at least one of the two needs to be 
# True)
def build_tfidf(list_of_jokes, include_title, include_post, n_feats, min_df = 10, max_df = 0.8):
    tfidf_vec = TfidfVectorizer(input='content', decode_error=u'ignore', strip_accents=u'unicode',
                                analyzer=u'word',max_features=n_feats,stop_words='english',
                                norm=u'l2',min_df=min_df,max_df=max_df,lowercase=True,vocabulary=None)
    data = [(dt['title'] + ' ' + dt['post'] if include_post else dt['title']) if include_title else 
            (dt['post'] if include_post else '') for dt in list_of_jokes]
    doc_by_vocab_sparse = tfidf_vec.fit_transform(data)
    # doc_by_vocab = doc_by_vocab_sparse.toarray()   # <-- Need?

    # Construct a inverted map from feature index to feature value (word) for later use
    index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}
    
    # return sparse tfidf matrix, and mapping showing the word's index to the word itself
    return (doc_by_vocab_sparse, index_to_vocab)

In [67]:
def filter_title_post(list_of_jokes):
    return [(dt['title'] + ' ' + dt['post'] if include_post else dt['title']) if include_title else 
            (dt['post'] if include_post else '') for dt in list_of_jokes]


In [68]:
#TESTS
d = [7]*6
print d
d[1] = 6
print d


n_feats = 5000
j = [{u'title': u'this is a title', u'post': u'punchline_e'},{u'title': u'this is a title2', u'post': u'punchline_d'},
     {u'title': u'this is a title3', u'post': u'punchline_c'},
     {u'title': u'this is a title4', u'post': u'punchline_b'},{u'title': u'this is a title5', u'post': u'punchline_a'}]
print build_tfidf(j, True, True, n_feats, min_df = 0, max_df = 1)
print '\n'
print build_tfidf(j, True, False, n_feats, min_df = 0, max_df = 1)
print '\n'
print build_tfidf(j, False, True, n_feats, min_df = 0, max_df = 1)
print '\n'
# Causes runtime error b/c there is clearly no text to consider at all.
#print build_tfidf(j, False, False, n_feats, min_df = 0, max_df = 1) 

[7, 7, 7, 7, 7, 7]
[7, 6, 7, 7, 7, 7]
(<5x10 sparse matrix of type '<type 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>, {0: u'punchline_a', 1: u'punchline_b', 2: u'punchline_c', 3: u'punchline_d', 4: u'punchline_e', 5: u'title', 6: u'title2', 7: u'title3', 8: u'title4', 9: u'title5'})


(<5x5 sparse matrix of type '<type 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>, {0: u'title', 1: u'title2', 2: u'title3', 3: u'title4', 4: u'title5'})


(<5x5 sparse matrix of type '<type 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>, {0: u'punchline_a', 1: u'punchline_b', 2: u'punchline_c', 3: u'punchline_d', 4: u'punchline_e'})




In [69]:
#TESTING
print get_inverted_index(j, False, True)

defaultdict(<type 'list'>, {u'punchline_d': [(1, 1)], u'punchline_e': [(0, 1)], u'punchline_a': [(4, 1)], u'punchline_b': [(3, 1)], u'punchline_c': [(2, 1)]})
